<a href="https://colab.research.google.com/github/sheaDurgin/TMAssignment4/blob/main/step2_base_weight_decay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
!pip install accelerate -U
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
import numpy as np

In [2]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [3]:
def tokenize_data(X, y, max_length=512):
    encodings = tokenizer(X, padding=True, truncation=True, max_length=max_length)
    return CustomDataset(encodings, y)

def get_tsv_data(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df['lyric'] = df['lyric'].str.lower().str.replace('[^\w\s]', '')

    X = df['lyric'].tolist()
    y = df['genre'].tolist()

    return X, y

In [4]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
label_encoder = LabelEncoder()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
train_file_path = 'train.tsv'
val_file_path = 'validation.tsv'
test_file_path = 'test.tsv'
X_train, y_train = get_tsv_data(train_file_path)
X_val, y_val = get_tsv_data(val_file_path)
X_test, y_test = get_tsv_data(test_file_path)
print("read data")
all_labels = y_train + y_val + y_test
label_encoder.fit(all_labels)
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

train_dataset = tokenize_data(X_train, y_train_encoded)
val_dataset = tokenize_data(X_val, y_val_encoded)
test_dataset = tokenize_data(X_test, y_test_encoded)

read data


In [6]:
model_path = 'distilroberta_results/'

In [7]:
# Early Stopping
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)
hyperparameters = [0.01, 0.001, 0.0001, 0.00001]
for param in hyperparameters:
    model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=len(set(y_train)))
    print(f"using {param}")
    # Define training arguments
    training_args = TrainingArguments(
        output_dir=model_path,
        learning_rate=2e-5,
        weight_decay=param,
        load_best_model_at_end = True,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=4,
        logging_dir='./logs',
        logging_steps=100,
        evaluation_strategy='epoch',
        save_strategy='epoch'
    )

    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[early_stopping],
    )

    # Train the model
    trainer.train()
    # trainer.save_model(model_path)

    # Evaluate the model
    test_results = trainer.predict(test_dataset)

    # Extract predicted labels
    predicted_labels = np.argmax(test_results.predictions, axis=1)

    # Decode predicted labels
    predicted_labels = label_encoder.inverse_transform(predicted_labels)

    # Calculate F1 score for each class and total F1 score
    f1_scores_per_class = f1_score(y_test, predicted_labels, average=None)
    total_f1_score = f1_score(y_test, predicted_labels, average='weighted')

    # Print F1 scores
    for label, f1_score_class in zip(label_encoder.classes_, f1_scores_per_class):
        print(f"F1 score for class {label}: {f1_score_class}")

    print(f"Total F1 score: {total_f1_score}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


using 0.01


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.145100,1.128743
2,0.995600,1.055203
3,0.867200,1.049267
4,0.781500,1.042320


F1 score for class blues: 1.0
F1 score for class country: 0.5454545454545454
F1 score for class metal: 0.6153846153846154
F1 score for class pop: 0.33333333333333337
F1 score for class rap: 0.8333333333333333
F1 score for class rock: 0.25
Total F1 score: 0.5962509712509712


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


using 1e-05


Epoch,Training Loss,Validation Loss
1,1.142400,1.095744
2,0.993600,1.041351
3,0.865400,1.034774
4,0.771100,1.035925


Checkpoint destination directory distilroberta_results/checkpoint-386 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distilroberta_results/checkpoint-772 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distilroberta_results/checkpoint-1158 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory distilroberta_results/checkpoint-1544 already exists and is non-empty. Saving will proceed but saved results may be invalid.


F1 score for class blues: 0.8333333333333333
F1 score for class country: 0.6
F1 score for class metal: 0.5454545454545454
F1 score for class pop: 0.6666666666666665
F1 score for class rap: 0.9090909090909091
F1 score for class rock: 0.28571428571428575
Total F1 score: 0.6400432900432899
